# MySQL Python connection

Some Python code to implement connection to the MySQL server and write attendance data to it.
We need to import the Python-MySQL driver to be used in connecting to the database and performing SQL statements(INSERT, SELECT)

A prerequisite is installation of the driver. This can be done by running the command below from a terminal:  
`pip install mysql-connector-python`

In [1]:
import mysql.connector
from mysql.connector import Error

## Database Configuration Parameters
Here we define constants to be used in connecting to the database:
* DB_HOST: the host server(using localhost for now, but will be changed when hosted)
* DB_PORT: the port on the host server where the database communicates
* DB_NAME: the database name
* USER_NAME & USER_PWD: database user credentials 

In [2]:
DB_HOST = "localhost"
DB_PORT ="3308"
DB_NAME = "gae"
USER_NAME = "root"
USER_PWD = ""

## Database Connection
This is where we connect to the connect using the parameters defined previously.
The connection attempt is surrounded in a try-catch block as the driver throws errors in case of failure.

In [10]:
def create_connection():
    connection = None
    try:
        connection = mysql.connector.connect(
            host=DB_HOST,
            port=DB_PORT,
            user=USER_NAME,
            passwd=USER_PWD,
            database=DB_NAME
        )
        print(f"Connection to MySQL {DB_NAME} database successful")
    except Error as e:
        print(f"Error: {e}")

    return connection

## SELECTION of student database ID

We first query the database to retrieve the student ID to be used in logging attendance.
Since the Face Recognition system uses national ID numbers as the identifier, we search for IDs using this parameter.

In [4]:
def fetch_student_id(connection, nat_id):
    query = "SELECT id FROM students WHERE national_id = %s"
    cursor = connection.cursor(buffered=True)
    cursor.execute(query, [nat_id])
    user_id = cursor.fetchone()[0]
    cursor.close()
    return user_id

## INSERTION of attendance record
Having retrieved the student's id, we can now INSERT their clocking in time to the `attendance` table.
We then print the `lastrowid` as feedback to show successful insertion.

In [8]:
def insert_student_attendance(nat_id, time_in):
    query = "INSERT INTO attendance(user_id, time_in) VALUES (%s, %s)"
    connection = create_connection()
    user_id = fetch_student_id(connection, nat_id)
    cursor = connection.cursor()
    attendance = (user_id, time_in)
    cursor.execute(query, attendance)
    print('Inserted record: ', cursor.lastrowid)
    connection.commit()
    cursor.close()
    connection.close()    

## Example

In [11]:

insert_student_attendance('36929828', '2023-02-20 09:50:49')

Connection to MySQL gae database successful
Inserted record:  15
